In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import src.config as config

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2025-03-26 11:41:49,547 INFO: Initializing external client
2025-03-26 11:41:49,547 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-26 11:41:55,627 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220769


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (15.46s) 


In [7]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_longitude,pickup_latitude,pickup_location
68880,2024-01-01 00:00:00+00:00,2,-87.721559,41.968069,Albany Park
372225,2024-01-01 01:00:00+00:00,0,-87.721559,41.968069,Albany Park
200573,2024-01-01 02:00:00+00:00,0,-87.721559,41.968069,Albany Park
350256,2024-01-01 03:00:00+00:00,3,-87.721559,41.968069,Albany Park
933432,2024-01-01 04:00:00+00:00,0,-87.721559,41.968069,Albany Park
...,...,...,...,...,...
684855,2025-03-26 12:00:00+00:00,0,-87.656804,41.949140,Wrigleyville
413265,2025-03-26 13:00:00+00:00,0,-87.656804,41.949140,Wrigleyville
869900,2025-03-26 14:00:00+00:00,0,-87.656804,41.949140,Wrigleyville
884872,2025-03-26 15:00:00+00:00,0,-87.656804,41.949140,Wrigleyville


In [24]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28,
    step_size=23
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f"{features_and_target.shape=}")

100%|██████████| 95/95 [00:20<00:00,  4.74it/s]


features_and_target.shape=(41990, 677)


In [25]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1)).tz_localize("UTC")

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date=cutoff_date,
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(39140, 676)
y_train.shape=(39140,)
X_test.shape=(2850, 676)
y_test.shape=(2850,)


In [26]:
import lightgbm as lgb

past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.135763 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 170917
[LightGBM] [Info] Number of data points in the train set: 39140, number of used features: 672
[LightGBM] [Info] Start training from score 7.620235


LGBMRegressor()

In [27]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=1.8386


In [32]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(model, MODELS_DIR / 'model.pkl')

['C:\\Users\\joral_08cedew\\chicago_taxi_demand_predictor\\models\\model.pkl']

In [33]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train_only_numeric)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [34]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor",
    input_example=X_train_only_numeric.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR/'model.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1328 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3360 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60583 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220769/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)